In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [44]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [45]:
#COMPOSERS LOADING
import pandas as pd

df = pd.read_csv('../../data/processed/composers/wikipedia.csv', index_col=0)

cursor = conn.cursor()
for idx,row in df.iterrows():
    insert = 'INSERT INTO COMPOSERS (NAME, COUNTRY, URL, URI) VALUES ('
    values = "'" + row['composer'].replace("'", "''") + "', '" + row['country'].replace("'", "''") + "', '" + row['composer_url'] + "', '" + row['composer_key'] + "'); "
    commit = 'COMMIT;'    
#     print(insert + values + commit)
    cursor.execute(insert + values + commit)

In [ ]:
#COMPOSERS LOADING
import pandas as pd

df = pd.read_csv('../../data/processed/composers/wikipedia.csv', index_col=0)

cursor = conn.cursor()
for idx,row in df.iterrows():
    insert = 'INSERT INTO COMPOSERS (NAME, COUNTRY, URL, URI) VALUES ('
    values = "'" + row['composer'].replace("'", "''") + "', '" + row['country'].replace("'", "''") + "', '" + row['composer_url'] + "', '" + row['composer_key'] + "'); "
    commit = 'COMMIT;'    
    print(insert + values + commit)
    cursor.execute(insert + values + commit)

In [ ]:
from bs4 import BeautifulSoup
import requests
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

In [167]:
import numpy as np
cw_df = pd.read_csv('../../data/processed/works/wikipedia.csv', index_col=0)
cwm_df = pd.read_csv('../../data/processed/works/more_works.csv', index_col=0)
cwm_df['composer_url'] = '/'
cwm_df['composer_key'] = cwm_df.apply(lambda row: row['composer_url'].split('/')[-1], axis=1)
cwm_df['work_key'] = cwm_df.apply(lambda row: row['work_url'].split('/')[-1], axis=1)
cwm_df = cwm_df.replace(np.nan, '', regex=True)

cwm_df = cwm_df.merge(cw_df, left_on=['work_key'], right_on=['work_key'], how='left')
cwm_df = cwm_df[['composer_y', 'work_y', 'composer_url_y', 'work_url_x', 'composer_key_y', 'work_key']]
cwm_df = cwm_df.rename(index=str, columns={"composer_y": "composer", "work_y": "work", "composer_url_y": "composer_url", "work_url_x": "work_url", "composer_key_y": "composer_key"})

#There are records that have not been matched and need to be updated before loading
miss_df = cwm_df[cwm_df['composer'].isna()]

In [168]:
#Match missing works to their composers before concatenating.
import pandas as pd

cursor = conn.cursor()  
cursor.execute("SELECT distinct name, url, uri FROM composers c")
comp_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'url', 'uri'])

for idx,row in miss_df.iterrows():    
    url = row['work_url']
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    try:
        result = soup.find(attrs={'id': 'P86'}).find(attrs={'class': 'wikibase-snakview-value wikibase-snakview-variation-valuesnak'})
        composer_key = ''
        composer_key = result.find('a').get('title')
        composer_url = 'http://www.wikidata.org/wiki/' + composer_key
        composer_r  = requests.get(composer_url, headers=headers)
        composer_soup = BeautifulSoup(composer_r.text, "html5lib")    
        composer_name = composer_soup.find(attrs={'class': 'wikibase-title-label'}).get_text()        
        work_name = soup.find(attrs={'class': 'wikibase-title-label'}).get_text()     
        if comp_df[comp_df['uri']==composer_key].shape[0] != 0:
            row['composer'] = result.find('a').get_text()
            row['composer_url'] = composer_url
            row['composer_key'] = composer_key
            row['work'] = work_name
        else:
            insert = 'INSERT INTO COMPOSERS (NAME, URL, URI) VALUES ('
            values = "'" + composer_name + "', '" + composer_url + "', '" + composer_key + "'); "
            commit = 'COMMIT;'
            print(insert + values + commit)            
    except:
        if row['work_url'] == 'http://www.wikidata.org/wiki/Q7144946':
            row['composer'] = 'Paula M. Kimper'
            row['composer_url'] = 'http://www.wikidata.org/wiki/P000001'
            row['composer_key'] = 'P000001'
            row['work'] = 'Patience and Sarah'            
        else:
            print(composer_key + '---' + url)

In [170]:
mapped_df = cwm_df[~cwm_df['composer'].isna()]

print(cw_df.shape)
print(cwm_df.shape)
print(mapped_df.shape)
print(miss_df.shape)

#Concatenate all mapped works into one DF
ref_df = pd.concat([cw_df, mapped_df, miss_df])
ref_df.shape

(3976, 6)
(599, 6)
(501, 6)
(98, 6)


(4575, 6)

In [176]:
#WORKS LOADING
import pandas as pd

cursor = conn.cursor()
for idx,row in ref_df.iterrows():
    insert = 'INSERT INTO WORKS (NAME, URL, URI, COMPOSER_URI) VALUES ('
    values = "'" + row['work'].replace("'", "''") + "', '" + row['work_url'] + "', '" + row['work_key'] + "', '" + row['composer_key'] + "'); "
    commit = 'COMMIT;'    
#     print(insert + values + commit)
    cursor.execute(insert + values + commit)

In [173]:
ref_df.head()

,composer,work,composer_url,work_url,composer_key,work_key
0,Aaron Copland,The Second Hurricane,http://www.wikidata.org/entity/Q192185,http://www.wikidata.org/entity/Q7762930,Q192185,Q7762930
1,Aaron Copland,The Tender Land,http://www.wikidata.org/entity/Q192185,http://www.wikidata.org/entity/Q4353383,Q192185,Q4353383
2,Aarre Merikanto,Juha,http://www.wikidata.org/entity/Q303051,http://www.wikidata.org/entity/Q4413332,Q303051,Q4413332
3,Aarre Merikanto,Helena,http://www.wikidata.org/entity/Q303051,http://www.wikidata.org/entity/Q15876167,Q303051,Q15876167
4,Abdylas Maldybaev,Ai-Churek,http://www.wikidata.org/entity/Q479615,http://www.wikidata.org/entity/Q18399378,Q479615,Q18399378
